# Stohastic simulation of the SIIS model

We introduce one case with strain 1, or 2, and count new cases from it (of strain 1, or 2). The simulation stops once that individual becomes healed. Initially there are Strain1, and Strain 2 cases in the population of total size N. The reported results are averages of NoSimulations repetitions.

In [3]:
import numpy as np

# Parameters
N = 10000 # Total population
SecondaryCasesList = [] #  List of No. of secondary cases at each simulation
NoSimulations = 10000

# Parameters
gam1 = 1.0
bet1 = 2.5
gam2 = 0.5
bet2 = 0.6
nu = 2.0

"""  We find here how much new cases of strain 1 will be, from introduced individual with strain 1 - K11""" 

Strain10 = 51 # Initial population with strain 1 (plus 1 tracked)
Strain20 = 100 # Initial population with strain 2

for j in range(NoSimulations): 
    Healed = False # Whether the tracked individual has healed (to stop simulation)
    Strain1 = Strain10  # Initialization at each new simulation
    Strain2 = Strain20    
    Susceptible = N - Strain1 - Strain2 # Initialize the number of susceptible
    NewCases = 0 # The secondary cases from the tracked individual
    while not Healed:
        # Simulation
        StatSum = gam1 * Strain1 + gam2 * Strain2 + bet1 * Strain1 * Susceptible / N + bet2 * Strain2 * Susceptible / N + nu * Strain1 * Strain2 / N        
        RndNum = np.random.random()   # Choose randomly the new state of the system - the trio (S, I1, I2)
        if RndNum < gam1 * Strain1 / StatSum: #  Healing from strain 1 of some individual
            if np.random.random() < 1.0 / Strain1: # Check whether the tracked individual has become healed
                Healed = True
                SecondaryCasesList.append(NewCases)            
            Strain1 -= 1
            Susceptible += 1              
        elif RndNum < (gam1 * Strain1 + gam2 * Strain2) / StatSum: #  Healing from strain 2 of some individual
            Strain2 -= 1
            Susceptible += 1
        elif RndNum < (gam1 * Strain1 + gam2 * Strain2 + bet1 * Strain1 * Susceptible / N) / StatSum: #  New case with strain 1
            if np.random.random() < 1.0 / Strain1: # Check whether the new case if from the tracked one
                NewCases += 1
            Strain1 += 1
            Susceptible -= 1
        elif RndNum < (gam1 * Strain1 + gam2 * Strain2 + bet1 * Strain1 * Susceptible / N + bet2 * Strain2 * Susceptible / N) / StatSum: #  New case with strain 2
            Strain2 += 1
            Susceptible -= 1
        else: #  Superinfection (from 2 to 1)
            Strain2 -= 1
            Strain1 += 1            
    if j % (NoSimulations // 10) == 0: # print 10 steps from the simulations (to see whether everything is OK)
        print(j, "  K11   ", sum(SecondaryCasesList) / len(SecondaryCasesList))

# final result for K11
print(" K 11 ", sum(SecondaryCasesList) / len(SecondaryCasesList))



"""  Introducing one individual with strain 2 to find K21 and K22"""

Strain10 = 50 # Initial population with strain 1 
Strain20 = 101 # Initial population with strain 2 (plus 1 tracked)
NoSimulations = 1000000 # Here we need more simulations for good statistics (since superinfection is rare)
SecondaryCasesList1 = [] 
SecondaryCasesList2 = [] 

for j in range(NoSimulations): # 
    Healed = False # Has the tracked individual become healed
    Superinfection = False # Account for whether the tracked individual has become superinfected
    Strain1 = Strain10   
    Strain2 = Strain20   
    Susceptible = N - Strain1 - Strain2
    NewCases1 = 0 # we track here secondary cases with strain 1 and strain 2 separately
    NewCases2 = 0
    while not Healed:        
        StatSum = gam1 * Strain1 + gam2 * Strain2 + bet1 * Strain1 * Susceptible / N + bet2 * Strain2 * Susceptible / N + nu * Strain1 * Strain2 / N
        RndNum = np.random.random()
        if RndNum < (gam1 * Strain1) / StatSum: #  Healing from strain 1 of some individual
            # Now check whether the tracked individual has become healed (and stop simulation)
            # it is one that has been superinfected
            if Superinfection and (np.random.random() < 1.0 / Strain1): 
                Healed = True
                SecondaryCasesList1.append(NewCases1)
                SecondaryCasesList2.append(NewCases2)                  
            Strain1 -= 1
            Susceptible += 1              
        elif RndNum < (gam1 * Strain1 + gam2 * Strain2) / StatSum: #  Healing from strain 2 of some individual
            # Check whether the tracked individual has become healed (and stop simulation)
            # it is one that has NOT been superinfected (it is possible to heal from 2 directly)
            if (not Superinfection) and (np.random.random() < 1.0 / Strain2):
                Healed = True              
                SecondaryCasesList1.append(NewCases1)
                SecondaryCasesList2.append(NewCases2)                                  
            Strain2 -= 1
            Susceptible += 1
        elif RndNum < (gam1 * Strain1 + gam2 * Strain2 + bet1 * Strain1 * Susceptible / N) / StatSum: #  New case with strain 1
            if Superinfection and (np.random.random() < 1.0 / Strain1): # Check whether the new case is from the tracked
                NewCases1 += 1
            Strain1 += 1
            Susceptible -= 1
        elif RndNum < (gam1 * Strain1 + gam2 * Strain2 + bet1 * Strain1 * Susceptible / N + bet2 * Strain2 * Susceptible / N) / StatSum: #  New case with strain 2
            if (not Superinfection) and (np.random.random() < 1.0 / Strain2): # Check whether the new case is from the tracked
                NewCases2 += 1
            Strain2 += 1
            Susceptible -= 1
        else: #  Superinfection
            if np.random.random() < 1.0 / Strain2:
                Superinfection = True
            Strain2 -= 1
            Strain1 += 1            
    if j % (NoSimulations // 10) == 0: # Print to check how the simulation goes on
        print(j, "  K21   ", sum(SecondaryCasesList1) / len(SecondaryCasesList1), "  K22  ", sum(SecondaryCasesList2) / len(SecondaryCasesList2)) 

# Final result for K21 and K22
print(" K 21 ", sum(SecondaryCasesList1) / len(SecondaryCasesList1))
print(" K 22 ", sum(SecondaryCasesList2) / len(SecondaryCasesList2))



0   K11    0.0
1000   K11    2.3896103896103895
2000   K11    2.4782608695652173
3000   K11    2.4131956014661777
4000   K11    2.3731567108222946
5000   K11    2.3817236552689462
6000   K11    2.3857690384935846
7000   K11    2.371232681045565
8000   K11    2.36920384951881
9000   K11    2.360071103210754
 K 11  2.3574
0   K21    0.0   K22   0.0
100000   K21    0.33914660853391465   K22   0.8795412045879542


KeyboardInterrupt: 

# Theoretical calculations

Here we find the theoretical values of the replacement numbers - in fact the secondary cases at various state-at-infection, resulting from one individual in certain state-at-infection

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# definition of the ODE for the SIIS model, where I is the strain 1, while J is the strain 2
# b is short for beta, while g is for gamma
def SIJ(y, t, b1, g1, b2, g2, nu):
    S, I, J = y
    dydt = [g1 * I + g2 * J - (b1 * I + b2 * J) * S, (b1 * S - g1 + nu * J) * I, (b2 * S - g2 - nu * I) * J]
    return dydt

# This function determines the integral of I_1(t) from 0 to t, 
# from given list of values of I_1 at discrete points separated at dT
def I1Integral(I1, dT):
    N = len(I1)
    Integ = np.zeros(N)
    for i in range(N - 1):
        Integ[i + 1] = Integ[i] + 0.5 * dT * (I1[i + 1] + I1[i])
    return Integ    

# This function is the list of probabilities p21 for superinfection at different moments t
def Superinfection(dT, g2, nu, I1):
    N = len(I1)
    IntegI1 = I1Integral(I1, dT)    
    p21 = np.zeros(N)
    for i in range(N):
        p21[i] = ni * I1[i] * np.exp(-g2 * dT * i - nu * IntegI1[i])
    return p21

# This function gives the list of survival probabilities of infection of the individual 
# with strain 2 (neither healed nor superinfected)
def ProbSurv2(dT, g2, ni, I1):
    N = len(I1)
    IntegI1 = I1Integral(I1, dT)    
    p2 = np.zeros(N)
    for i in range(N):
        p2[i] = np.exp(-g2 * dT * i - ni * IntegI1[i]) 
    return p2
        
# Integral formula for K_11 - integral from 0 to infinity 
# In practice it is some finite but large enough value so that 
# healing of the tracked is nearly certain
def K11Int(b1, g1, S, dT):
    NoPoints = len(S)
    K11 = 0.5 * S[0] # initial values (it is omitted the exponential term exp(0) = 1
    for i in range(NoPoints - 2):
        K11 += S[i + 1] * np.exp(- g1 * dT * (i + 1))
    K11 += 0.5 * S[NoPoints - 1] * np.exp(- g1 * dT * (NoPoints - 1))   
    K11 *= (b1 * dT)
    return K11

# This function finds K_11 (needed for K21) but starting from arbitrary initial time t 
# (it should be an integral from t to infinity)
# The function returns list of the integrals for all t from 0 to T/2, 
# where T is the total duration of the integrations
# (when we consider that the introduced case has healed with probability -> 1)
# We apply a trick to speed up the simulations: 
# Find first the last value (for t = T/2) and then go backward to 0
def K11IntOfT(b1, g1, S, dT):
    # First find the last value for t = T/2
    NoPoints = len(S)
    N = NoPoints // 2
    SInt = np.zeros(NoPoints)
    K11Int = np.zeros(NoPoints)
    IntegSum = 0.5 * S[N] * np.exp(-g1 * dT * N)
    for i in range(1, N - 1):
        IntegSum += S[N + i] * np.exp(-g1 * dT * (N + i))
    IntegSum += 0.5 * S[2 * N - 1] * np.exp(-g1 * dT * (2 * N - 1))    
    SInt[N] = IntegSum * dT * b1
    K11Int[N] = SInt[N] * np.exp(g1 * dT * N)

    # Now go backward until t = 0
    for i in range(N - 1, -1, -1):
        # first find the value of the integral of S(t) * exp(-gamma_1 * t)
        SInt[i] = SInt[i + 1] + dT * 0.5 * b1 * (S[i + 1] * np.exp(-g1 * dT * (i + 1)) + S[i] * np.exp(-g1 * dT * (i)))        
        K11Int[i] = SInt[i] * np.exp(g1 * dT * i)
    return K11Int

# This function finds the expected new cases of strain 1, from introdiced individual with strain 2 
# (after becoming superinfected)
def K21Int(b1, g1, g2, ni, S, I1, dT):
    P21 = Superinfection(dT, g2, ni, I1)   # list of probabilities of superinfection at all moments t
    K11ofT = K11IntOfT(b1, g1, S, dT)
    N = len(S) // 2
    K21 = 0.5 * P21[0] * K11ofT[0]
    for i in range(N - 2):
        K21 += P21[i + 1] * K11ofT[i + 1]
    K21 += 0.5 * P21[N - 1] * K11ofT[N - 1]        
    K21 *= dT
    return K21


# This funciton finds the expected number of secondary cases of the same strain 2
def K22Int(b2, g2, ni, S, I1, dT):  
    N = len(S)
    p2 = ProbSurv2(dT, g2, ni, I1)
    K22 = 0.5 * S[0] * p2[0]
    for i in range(1, N - 1):
        K22 += S[i] * p2[i]
    K22 += 0.5 * S[N - 1] * p2[N - 1]        
    K22 *= dT * b2
    return K22
            

I0 = 0.005
J0 = 0.01
S0 = 1 - I0 - J0

g1 = 1.0
b1 = 2.5
g2 = 0.5
b2 = 0.6
nu = 2.0

y0 = [S0, I0, J0]
Duration = 100
Steps = 100
dT = 1 / Steps

t = np.linspace(0, Duration, 1 + Steps * Duration)
SIIS_model = odeint(SIJ, y0, t, args=(b1, g1, b2, g2, nu))

plt.plot(t, SIIS_model[:, 0], 'b', label='S(t)')
plt.plot(t, SIIS_model[:, 1], 'g', label='I1(t)')
plt.plot(t, SIIS_model[:, 2], 'r', label='I2(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.xlim(0,20)
plt.grid()
plt.show()


K22 = K22Int(b2, g2, nu, SIIS_model[:, 0], SIIS_model[:, 1], dT)
K21 = K21Int(b1, g1, g2, nu, SIIS_model[:, 0], SIIS_model[:, 1], dT)
K11 = K11Int(b1, g1, SIIS_model[:, 0], dT)
print("  for initial infected fractions ", I0, J0, "  expected new cases are K11, K21, K22 ", K11, K21, K22)
